## 집값 예측 훈랸

In [4]:
from tensorflow.keras.models import load_model
import pandas as pd
import time

kaggle_test = pd.read_csv('./data/house_test.csv')

# 카테고리형 변수를 0,1 가진 변수로 변환 (원-핫)
kaggle_test = pd.get_dummies(kaggle_test)

# 결측치를 전체 칼럼의 평균으로 대체
kaggle_test = kaggle_test.fillna(kaggle_test.mean())

In [12]:
# 집 값을 제외한 나머지 열을 저장합니다.
cols_kaggle = ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF']
K_test = kaggle_test[cols_kaggle]
K_test

,OverallQual,GrLivArea,GarageCars,GarageArea,TotalBsmtSF
0,5,896,1.0,730.0,882.0
1,6,1329,1.0,312.0,1329.0
2,5,1629,2.0,482.0,928.0
3,6,1604,2.0,470.0,926.0
4,8,1280,2.0,506.0,1280.0
...,...,...,...,...,...
1454,4,1092,0.0,0.0,546.0
1455,4,1092,1.0,286.0,546.0
1456,5,1224,2.0,576.0,1224.0
1457,5,970,0.0,0.0,912.0


In [15]:
model = load_model("./data/model/Ch15-house.hdf5")
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 10)             │            60 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │           330 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 40)             │         1,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            41 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,673 (6.54 KB)

 Trainable params: 1,671 (6.53 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [16]:
# ID와 예측값이 들어갈 빈 리스트 생성
ids = []

# 예측 값 생성
Y_prediction = model.predict(K_test).flatten()
for i in range(len(K_test)):
    id = kaggle_test['Id'][i]
    prediction = Y_prediction[i]
    ids.append([id, prediction])

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 533us/step


In [18]:
# 테스트 결과의 저장 환경을 설정합니다.
timestr = time.strftime("%Y%m%d-%H%M%S")
filename = str(timestr) # 파일명을 연월일-시분초로 정합니다.
outdir = './'

In [22]:
df = pd.DataFrame(ids, columns = ["Id", "SalePrice"])
df.to_csv(str(outdir + filename + '_submisstion.csv'), index = False)